In [444]:
import requests
requests.packages.urllib3.disable_warnings() 
import xml.etree.ElementTree as ET
import re
import urllib.parse
import sys, getopt

# extract endpoints

In [434]:
def get_paths_methods(SITEMAP_FILE_PATH):
    tree = ET.parse(SITEMAP_FILE_PATH)
    root = tree.getroot()
    titer = tree.iter
    paths =root.findall('.//path')
    paths_list = [paths[i].text for i in range(len(paths))]
    methods =root.findall('.//method')
    methods_list = [methods[i].text for i in range(len(methods))]
    return paths_list, methods_list

In [435]:
def cleaning_paths(paths,methods):
    blacklist = ["/login","signin","/logout","signout"]
    lenght = len(paths)
    
    clean_paths, clean_methods = [], []
    for i in range(0,lenght):
        if paths[i] not in blacklist and paths[i] not in clean_paths:
            clean_paths.append(paths[i])
            clean_methods.append(methods[i])
    return clean_paths, clean_methods

In [436]:
def get_csrf_token(response, look_in="html"):
    if look_in == "html":
        pattern = re.compile(r'(?<=csrf\" value=\")(.*)(?=\")')
        token = pattern.search(response).group(0)
    # the following code is generated by GitHub Copilot and has to be authenticated
    elif look_in == "xml":
        root = ET.fromstring(response)
        token = root.find('.//csrf').text
    elif look_in == "json":
        token = response["csrf"]
    elif look_in == "http":
        token = response.cookies["csrf"]
    return token

In [437]:
def login(login_url, username, password, look_in="html"):
    """_summary_
    
    Not implemented yet
    
    """
    
    session = requests.Session()
    
    if look_in == "html":
        response =session.get(login_url, verify=False).text
        csrf = get_csrf_token(response)
    
    post_data=f"username={urllib.parse.quote(username)}&password={urllib.parse.quote(password)}"

    response2 = session.post(login_url,verify=False,data=post_data)
    
    if response2.status_code != 200:
        print("Login error!\nCheck creds")
        return None
    
    return session


In [438]:
def send_requests(paths, methods, url,COOKIE):
    if (len(paths)!= len(methods)):
        return print("lenght mismatch!")
    lenght = len(paths)
    responses = [0] * lenght
    
    
    for i in range(0,lenght):
        if(methods[i] == "GET"):
            responses[i] = requests.get(url=f"{url}{paths[i]}", verify=False, cookies=COOKIE)
        if(methods[i] == "POST"):
            responses[i] = requests.post(url=f"{url}{paths[i]}", verify=False, cookies=COOKIE)
        if(methods[i] == "PUT"):
            responses[i] = requests.put(url=f"{url}{paths[i]}", verify=False, cookies=COOKIE)
        if(methods[i] == "DELETE"):
            responses[i] = requests.delete(url=f"{url}{paths[i]}", verify=False, cookies=COOKIE)
            
    return responses


In [439]:
def handle_result(result):
    codes =[]
    for response in result:
        codes.append(response.status_code)
        
    code_set = set(codes)
    code_results = [[0]*len(code_set),[]]

    code_dic = {}

    for code in code_set:
        code_dic[code] = []
        print(f"status [{code}] count: {codes.count(code)}")
        
    for response in result:
        code_dic[int(response.status_code)].append(response)
            
    return code_dic

In [440]:
def save_to_file(fileName, result_dic, username ,password, onlyPath=True):
    file = open(fileName,"w")
    file.write(f"Username: {username}, Password: {password}\n")
    for k in result_dic:
        for result in result_dic.get(k):
            file.write(f"{result.request.method}  {result.request.path_url}\t[{k}]\n")
            
            if (not onlyPath):
                file.write(f"Response:\n{result.text}\n")
                
        file.write("\n==================================\n")
    file.close()

# Main

In [441]:
URL =""
login_url =""

PATHS , METHODS = get_paths_methods()
PATHS, METHODS = cleaning_paths(PATHS, METHODS)
COOKIE = {"":""}
username = ""
password = ""

output_file = f"{URL}_output.txt"

In [ ]:
# main
session = login(login_url,username,password)
if session == None:
    exit()
result = send_requests(PATHS, METHODS, URL, COOKIE)

result_dic = handle_result(result)
save_to_file(output_file, result_dic, username ,password, onlyPath=True)

In [ ]:
argumentList = sys.argv[1:]
if len(argumentList) < 1:
    print("""Usage: apiauth -u <URL> -a <API FILE> -o <OUTPUT FILE> (optins)\
    help: apiauth -h 
    """)
    exit()

options = "hmo:"
 
# Long options
long_options = ["Help", "My_file", "Output="]
 
try:
    # Parsing argument
    arguments, values = getopt.getopt(argumentList, options, long_options)
        
    for currentArgument, currentValue in arguments:
 
        if currentArgument in ("-h", "--help"):
            print ("Displaying Help")
             
        elif currentArgument in ("-u", "--url"):
            print ("Displaying file_name:", sys.argv[0])
             
        elif currentArgument in ("-o", "--Output"):
            print (("Enabling special output mode (% s)") % (currentValue))

except getopt.error as err:
    print (str(err))

